In [1]:
import json
from enum import Enum

class AksharType(Enum):
    PUREVOWEL = 1
    MAATRAA = 2
    HALANT = 3
    HALF = 4
    CONSONANT = 5
    CHANDRABINDU = 6
    BOTTOMBINDI = 7
    OTHER = 8
    EMPTY = None

In [2]:
class Akshar:
    def __init__(self, s, code, aType, mainC, vowel, rhy, urhy):
        self.s = str(s)
        self.code = int(code)
        self.aType = aType
        self.mainC = mainC
        self.vowel = vowel
        self.rhy = int(rhy)
        self.urhy = int(urhy)

    def setRhythm(self):
        self.rhy = Akshar.vowelRhythm(a.vowel)

    def json(self):
        s = "{s:'"+self.s+"',code:"+str(self.code)+",type:'"+self.aType.name+"',"
        s = s + "mainC:'"+self.mainC+"',vowel:'"+self.vowel+"',rhy:"+str(self.rhy)+",urhy:"+str(self.urhy)+"}"
        return s

    def isHindi(cd):
        return ((cd >= 2305) and (cd <= 2399))

    def isPureVowel(cd):
        return ((cd >= 2309) and (cd <= 2324))
      
    def isMaatraaVowel(cd):
        return ((cd >= 2366) and (cd <= 2380))

    def isChandraBindu(c):
        return (ord(c) == 2305)

    def isBindu(c):
        return (ord(c) == 2306)

    def isHalant(cd):
        return (cd == 2381)

    def isBottomBindi(cd):
        return (cd == 2364)

    def vowelRhythm(c):
        match c:
            case 'अ':
                return 1
            case 'आ':
                return 2
            case 'इ':
                return 1
            case 'ई':
                return 2
            case 'उ':
                return 1
            case 'ऊ':
                return 2
            case 'ए':
                return 2
            case 'ऐ':
                return 2
            case 'ओ':
                return 2
            case 'औ':
                return 2
            case 'ऑ':
                return 2
            case 'ऋ':
                return 1
            case _:
                return 0

    def maatraaToVowel(c):
        match c:
            case 'ा':
                return 'आ'
            case 'ि':
                return 'इ'
            case 'ी':
                return 'ई'
            case 'ु':
                return 'उ'
            case 'ू':
                return 'ऊ'
            case 'े':
                return 'ए'
            case 'ै':
                return 'ऐ'
            case 'ो':
                return 'ओ'
            case 'ौ':
                return 'औ'
            case 'ॉ':
                return 'ऑ'
            case 'ृ':
                return 'ऋ'
            case _:
                return c

    def compare(a,b):
        return True if a.str == b.str else False

    def vowelCompare(a,b):
        return True if a.vowel == b.vowel else False

In [3]:
emptyAkshar = Akshar("",0,AksharType.EMPTY,'','',0,0)
space = Akshar (" ", 32, AksharType.OTHER, ' ', ' ', 0, 0)

In [16]:
class AksharAddl:
    def unitsLast(akshars):
        if not akshars:
            return emptyAkshar
        else:
            return akshars[-1]

    def processChar(c):
        cd = ord(c)
        m = 0
        a = Akshar(str(c),ord(c), AksharType.OTHER, c, vowel = c, rhy = 0, urhy = 0)
        newRhythm = 0
        if Akshar.isPureVowel(cd): 
            newRhythm = Akshar.vowelRhythm(a.vowel)
        elif Akshar.isMaatraaVowel(cd):
            newRhythm = Akshar.vowelRhythm(Akshar.maatraaToVowel(a.vowel))
        else:
            newRhythm = 0
        aRhythm = Akshar.vowelRhythm('अ')
        
        if Akshar.isHindi(cd):
            if Akshar.isPureVowel(cd):
                return Akshar(str(c), ord(c), AksharType.PUREVOWEL, c, c, newRhythm, newRhythm)
            elif Akshar.isMaatraaVowel(cd):
                return Akshar(str(c), ord(c), AksharType.MAATRAA, c, c, newRhythm, newRhythm)
            elif Akshar.isBindu(c):
                return Akshar(str(c), ord(c), AksharType.HALF, c, c, 0, 0)
            elif Akshar.isHalant(cd):
                return Akshar(str(c), ord(c), AksharType.HALANT, c, c, 0, 0)
            elif Akshar.isChandraBindu(c):
                return Akshar(str(c), ord(c), AksharType.CHANDRABINDU, c, c, 0, 0)
            elif Akshar.isBottomBindi(cd):
                return Akshar(str(c), ord(c), AksharType.BOTTOMBINDI, c, c, 0, 0) 
            else:
                return Akshar(str(c), ord(c), AksharType.CONSONANT, c, 'अ', aRhythm, aRhythm)
        else:
            return a

    def mergeBottomBindi(aB,aC,aPart):
        if (aB.rhy > 0): 
            return aPart 
        else:
            return Akshar(aPart.s, aPart.code, aPart.aType, aPart.mainC, aC.vowel, aC.rhythm, aC.rhythm)

    def mrgMaatraaCons(aM, aC):
        aNew = Akshar(aC.str ++ aM.str, aC.code, ac.aType, aC.mainC, aM.vowel, aM.rhythm, aM.rhythm)
        if (aC.aksharType == AksharType.CONSONANT):
            match aM.aksharType: 
                case AksharType.MAATRAA:
                    (True, aNew)
                case AksharType.HALANT:
                    return (True, Akshar(aC.s ++ aM.s, aC.code, AksharType.HALF, aC.mainC, aM.vowel, aM.rhythm, aM.rhythm))
                case AksharType.BOTTOMBINDI:
                    return (True, AksharAddl.mergeBottomBindi(aM,aC,aNew))
                case _:
                    return (False, aM)
        else:
            return (False, aM)

    def calcHalfAksharRhythm(ac, ap, an):
        if (ac.aksharType != AksharType.HALF):
            return ac
        elif (((ac.mainChar == 'म') and (an.mainChar == 'ह')) or ((ac.mainChar == 'न') and (an.mainChar == 'ह'))):
            return ac
        elif ((ap.rhy == 1) and (ap.aksharType != AksharType.Half)):
            return Akshar(ac.s, ac.code, ac.AksharType, ac.mainC, ac.vowel, 1, 1)
        elif ((ap.rhy == 2) and (an.rhy == 2)):
            return ac
        else:
            return ac

In [9]:
def processChars(chars):
    return [AksharAddl.processChar(c) for c in chars]

processedChars = processChars("हम")
for a in processedChars:
    print(a.json())

{s:'ह',code:2361,type:'CONSONANT',mainC:'ह',vowel:'अ',rhy:1,urhy:1}
{s:'म',code:2350,type:'CONSONANT',mainC:'म',vowel:'अ',rhy:1,urhy:1}
